<a href="https://colab.research.google.com/github/Dannykik/ULPGL_work/blob/main/L3_work/detection_anomalie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Chemin du fichier (Google Drive)
file_path = "/content/drive/MyDrive/Mon_memoire_L3/UCSD_Anomaly_Dataset.tar.gz"

# Extraire l'archive dans /content
!tar -xvzf "$file_path" -C /content

In [ ]:
import os
os.listdir('/content')

In [ ]:
os.listdir('/content/UCSD_Anomaly_Dataset.v1p2')

In [ ]:

train_path = '/content/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Train'
print(os.listdir(train_path)[:5])

In [ ]:
test_path = '/content/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test'
print(os.listdir(test_path)[:5])

#Activer le GPU (vérification)

In [ ]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

#Charger les images (proprement)

In [ ]:
import os
import cv2
import numpy as np

IMG_SIZE = 128

def load_images_from_folder(folder):
    images = []
    for subfolder in sorted(os.listdir(folder)):
        sub_path = os.path.join(folder, subfolder)
        if os.path.isdir(sub_path):
            for file in sorted(os.listdir(sub_path)):
                if file.endswith('.tif'):
                    img_path = os.path.join(sub_path, file)
                    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                    img = img / 255.0
                    images.append(img)
    return np.array(images)

train_folder = '/content/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Train'
train_images = load_images_from_folder(train_folder)

print("Nombre d'images d'entraînement :", train_images.shape)

#Construire l’Autoencoder CNN

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D

train_images = train_images.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

input_img = Input(shape=(IMG_SIZE, IMG_SIZE, 1))

# Encoder
x = Conv2D(32, (3,3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2,2), padding='same')(x)
x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2,2), padding='same')(x)

# Decoder
x = Conv2D(64, (3,3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2,2))(x)
x = Conv2D(32, (3,3), activation='relu', padding='same')(x)
x = UpSampling2D((2,2))(x)
decoded = Conv2D(1, (3,3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

autoencoder.summary()


#Entraînement

In [ ]:
autoencoder.fit(
    train_images,
    train_images,
    epochs=10,
    batch_size=16,
    shuffle=True
)

#Détection des anomalies

In [ ]:
test_folder = '/content/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test'
test_images = load_images_from_folder(test_folder)
test_images = test_images.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

print("Nombre d'images de test :", test_images.shape)


#Reconstruction avec l’autoencoder

In [ ]:
reconstructed = autoencoder.predict(test_images)

#Calculer l’erreur de reconstruction

In [ ]:
mse = np.mean(np.power(test_images - reconstructed, 2), axis=(1,2,3))

#Définir un seuil

In [ ]:
threshold = np.mean(mse) + 3*np.std(mse)
print("Seuil d'anomalie :", threshold)

#Identifier les anomalies

In [ ]:
anomalies = mse > threshold
print("Nombre d'anomalies détectées :", np.sum(anomalies))

#Visualiser quelques résultats

In [ ]:
import matplotlib.pyplot as plt

for i in range(5):
    plt.figure(figsize=(6,3))
    # Image originale
    plt.subplot(1,2,1)
    plt.imshow(test_images[i].reshape(IMG_SIZE,IMG_SIZE), cmap='gray')
    plt.title("Originale")
    plt.axis('off')

    # Image reconstruite
    plt.subplot(1,2,2)
    plt.imshow(reconstructed[i].reshape(IMG_SIZE,IMG_SIZE), cmap='gray')
    plt.title(f"Reconstruit\nErreur={mse[i]:.4f}")
    plt.axis('off')

    plt.show()

In [ ]:
# Sauvegarder
autoencoder.save('/content/autoencoder_ucsd.h5')

In [ ]:
drive.mount('/content/drive')

autoencoder.save('/content/drive/MyDrive/Mon_memoire_L3/autoencoder_ucsd.keras')

In [ ]:
autoencoder.save('/content/drive/MyDrive/Mon_memoire_L3/autoencoder_ucsd.h5')

In [ ]:
!pip install ultralytics

In [ ]:
import torch
print("GPU disponible :", torch.cuda.is_available())


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from ultralytics import YOLO

# Modèle pré-entraîné sur COCO
model = YOLO("yolov8n.pt")   # version légère (rapide)

In [ ]:
!unzip "/content/drive/MyDrive/Mon_memoire_L3/New weapon detection/weapon detection.v1i.yolov8.zip" -d /content/weapon_dataset


In [8]:
import os

base_path = "/content/drive/MyDrive/Mon_memoire_L3/New weapon dataset"
os.makedirs(base_path, exist_ok=True)
import zipfile

zip_path = "/content/drive/MyDrive/Mon_memoire_L3/New weapon detection/weapon detection.v1i.yolov8.zip"
extract_path = base_path

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extrait avec succès !")

KeyboardInterrupt: 

In [ ]:
model.train(
    data=base_path + "/data.yaml",
    epochs=100,
    imgsz=640,
    batch=16,
    optimizer="AdamW",
    lr0=0.001,
    weight_decay=0.0005,  # Régularisation L2
    patience=20,
    device=0,
    project=base_path,    # IMPORTANT : sauvegarde dans TON dossier
    name="weapon_detection_training"
)